In [4]:
# import necessary modules
from selenium import webdriver
from selenium.webdriver.common.by import By
import openai

In [5]:
from bs4 import BeautifulSoup

I'm using selenium here to simulate web browsing.
After much research into the OneTrust APIs it turns out they are access-controlled and require an OAuth Token from the company to change consent and preferences on behalf of the user. So, instead, I have chosen to focus on clicking the decline all button on behalf of the user.

In [395]:
url_dict = {}
# OPTIONS FOR SELENIUM


In [61]:
len("Do not sell or share my data")

28

My inspection of the onetrust website shows that the ID of the decline all button is "onetrust-reject-all-handler" and hence I can go in and click that button.

In [1]:
import openai
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import time
import re
# Set your OpenAI API key here
api_key = KEY
from openai import OpenAI
client = OpenAI(api_key = api_key)

    
def extract_text_elements(html):
    """
    Extracts and formats text elements from HTML for analysis.

    Parameters:
    - html (str): The HTML content of the webpage.

    Returns:
    - list of tuples: Each tuple contains the text and its associated tag or identifier.
    """
    soup = BeautifulSoup(html, 'html.parser')
    elements = {}
    for i in ['confirm_my_preferences', 'accept_all', 'reject_all', 'manage_my_preferences']:
        elements[i] = []
    for tag in soup.find_all(['button', 'a']):
        text = tag.get_text(strip=True)
        #if len(text) > 40:
        #    continue
        text_lower = text.lower()
        identifier = 'None'
        if tag.get('id') :
            identifier = tag.get('id')
        classifier = 'None'
        if tag.get('class') :
            classifier = tag.get('class')[0]
        if any([t in text_lower for t in ['accept', 'allow', 'agree', 'yes']]):
            elements['accept_all'].append((text, identifier, classifier))
        if any([t in text_lower for t in ['reject', 'disable', 'necessary', 'only', 'no', 'disagree', 'essential']]):
            elements['reject_all'].append((text, identifier, classifier))
        if any([t in text_lower for t in ['customize', 'customise', 'setting', 'preference', 'manage', 'no', 'cookie', 'don']]):
            elements['manage_my_preferences'].append((text, identifier, classifier))
        if any([t in text_lower for t in ['save', 'exit', 'close', 'confirm', 'choice', 'selected']]):
            elements['confirm_my_preferences'].append((text, identifier, classifier))
    print(elements)
    return elements

def predict_cookie_banner(elements):
    """
    Uses GPT to predict which elements are likely part of a cookie banner.

    Parameters:
    - elements (list of tuples): Texts and their associated identifiers from the webpage.

    Returns:
    - list of tuples: Predicted cookie banner texts and their identifiers.
    """
    button_to_prompt = {
                        'manage_my_preferences' : 'manage or customize my cookies and preferences on the site',
                        'reject_all' : 'reject all cookies or accept only necessary/essential cookies on the site',
                        'accept_all' : 'accept or allow all cookies on the site',
                        'confirm_my_preferences' : 'confirm and save my preferences after they have selected them on the site'
                       }
    
    
    responses = ''
    for button_type in button_to_prompt :
        prompts = [f"Option: [button_text: {text}] [button_id: {identifier}] [button_class: {classifier}] \n" for text, identifier, classifier in elements[button_type]]
        combined_prompt = "Which HTML button among the following corresponds to the button that lets the user " + button_to_prompt[button_type] + "? Here are the options: \n" + "\n".join(prompts)
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
        {"role": "system", "content": "You are a web developer, skilled at identifying what HTML buttons are in cookie banners and you will quote the HTML button(s) to click for the button types: manage_my_preferences, accept_all, reject_all, confirm_my_preferences, etc. \n \
        And you will return outputs in the following format: [button_type: insert_button_type_here] [button_text: insert_button_text_here] [button_id: insert_button_id_here] [button_class: insert_button_class_here] \n and if you cannot find a corresponding button, return 'None' for button_text and button_id"},
        {"role": "user", "content": combined_prompt}
          ]
        )
        print([thingy.message.content for thingy in response.choices])
        responses += ' ' + response.choices[0].message.content
    print(responses)
    return responses

def parse_response(response):
    button_type = re.findall(r'\[button_type: ([^]]*)\]', response)
    button_text = re.findall(r'\[button_text: ([^]]*)\]', response)
    button_id = re.findall(r'\[button_id: ([^]]*)\]', response)
    button_class = re.findall(r'\[button_class: ([^]]*)\]', response)
    returned_dict = {}
    for i in range(len(button_type)):
        if button_text != "None" and ((not (button_id[i] == "None" or button_id[i] == "N/A")) or (not (button_class[i] == "None") or (button_class[i] != "N/A"))) :
            returned_dict[button_type[i]] = {"text" : button_text[i],
                                            "id" : button_id[i],
                                            "class" : button_class[i]}
    return returned_dict
    

def what_to_click(driver, url):
    driver.get(url)
    print("managed to get the webpage")
    sleep_time = 3
    button_text = ""
    sleep(sleep_time)
    sleep_time += 1
    html_source = driver.page_source
    print("managed to get the page source")
    url_key = url.split('.')[1]
    if url_key not in url_dict :
        elements = extract_text_elements(html_source)
        print("managed to get the text elements")
        responses = predict_cookie_banner(elements)
        print("managed to use gpt to parse it")
        parsed_response = parse_response(responses)
        if 'manage_my_preferences' in parsed_response:
            # click the manage my preferences button
            click_button(driver, parsed_response['manage_my_preferences'])
            sleep(sleep_time)
            html_source = driver.page_source
            elements = extract_text_elements(html_source)
            responses = predict_cookie_banner(elements)
            parsed_response2 = parse_response(responses)
            parsed_response.update(parsed_response2)
        print("PARSED RESPONSE FOR WEBSITE: ", parsed_response)
        url_dict[url_key] = parsed_response
        
def click_button(driver, button_dict):
    button_text = button_dict['text']
    button_id = button_dict['id']
    button_class = button_dict['class']
    if button_id != "None":
        button = driver.find_element(By.ID, button_id)
        driver.execute_script("arguments[0].click();", button)
        print(f'Clicked element ID {button_id}')
    elif button_text != "None":
        button = driver.find_element(By.XPATH, f"//*[text()='{button_text}']");
        driver.execute_script("arguments[0].click();", button)
        print(f'Clicked element text {button_text}')
    else:
        button = driver.find_element(By.CLASS_NAME, button_class)
        driver.execute_script("arguments[0].click();", button)
        print(f'Clicked element class {button_class}')
        

In [84]:
urls = [
        #"https://dl.acm.org/doi/10.1145/3313831.3376321",
        #"https://www.onetrust.com",
        #"https://www.stackoverflow.com/questions/31212059/its-possible-to-do-an-xhr-call-and-render-the-output-with-selenium",
        #"https://www.cnet.com",
        #"https://www.chicagobooth.edu/offerings/taking-booth-classes",
        #"https://www.transfermarkt.com/al-nasr-riad/startseite/verein/18544",
        #"https://www.nytimes.com/",
        #"https://us.puma.com/us/en",
        "https://www.samsung.com",
        "https://www.jamieoliver.com",
        "https://european-union.europa.eu/index_en",
       ]
driver = webdriver.Firefox()
for u in urls:
    sleep(3)
    t0 = time.time()
    what_to_click(driver, u)
    t1 = time.time()
    print(u, " takes ", t1 - t0, " seconds to load and click the cookie banner \n\n\n")


managed to get the webpage
managed to get the page source
{'confirm_my_preferences': [('close', 'None', 'cookie-bar__close'), ('close menu', 'None', 'nv00-gnb__close-btn'), ('Save Energy', 'None', 'nv00-gnb__l2-menu-link'), ('Layer Popup Close', 'None', 'layer-popup__close'), ('Layer Popup Close', 'None', 'layer-popup__close'), ('Layer Popup Close', 'None', 'layer-popup__close'), ('Close Check Preferences', 'None', 'layer-popup__close'), ('Close', 'None', 'gnb-search__input-btn--close'), ('Close', 'None', 'gnb-search__btn--close'), ('Close', 'None', 'popup-video__btn-video-close'), ('Close', 'None', 'popup-video__btn-video-close'), ('Close', 'None', 'popup-video__btn-video-close'), ('NewGalaxy S24 Ultra, 512GB (Unlocked)Galaxy S24 Ultra, 512GB (Unlocked)Save up to $150.From $1269.99 before eligible trade-in.Buy now', 'None', 'showcase-card-tab-card__full-bleed-wrap'), ('Tab S9 FETab S9 FESave up to $400.From $349.99 before eligible trade-in.Buy now', 'None', 'showcase-card-tab-card__fu

In [85]:
url_dict

{'acm': {'manage_my_preferences': {'text': 'Preferences (5)',
   'id': 'CybotCookiebotDialogDetailBodyContentCookieContainerPreference',
   'class': 'CybotCookiebotDialogDetailBodyContentCookieContainerTypesSelected'},
  'reject_all': {'text': 'Use necessary cookies only',
   'id': 'CybotCookiebotDialogBodyLevelButtonLevelOptinDeclineAll',
   'class': 'CybotCookiebotDialogBodyButton'},
  'accept_all': {'text': 'Allow all cookies',
   'id': 'CybotCookiebotDialogBodyButtonAccept',
   'class': 'CybotCookiebotDialogBodyButton'}},
 'onetrust': {'manage_my_preferences': {'text': 'Customize Settings',
   'id': 'onetrust-pc-btn-handler',
   'class': 'None'},
  'reject_all': {'text': 'Disable All',
   'id': 'onetrust-reject-all-handler',
   'class': 'None'},
  'accept_all': {'text': 'Allow All',
   'id': 'onetrust-accept-btn-handler',
   'class': 'None'},
  'confirm_my_choices': {'text': 'Confirm My Choices',
   'id': 'None',
   'class': 'save-preference-btn-handler'},
  'confirm_my_preferences

In [83]:
url_dict = {'acm': {'manage_my_preferences': {'text': 'Preferences (5)',
   'id': 'CybotCookiebotDialogDetailBodyContentCookieContainerPreference',
   'class': 'CybotCookiebotDialogDetailBodyContentCookieContainerTypesSelected'},
  'reject_all': {'text': 'Use necessary cookies only',
   'id': 'CybotCookiebotDialogBodyLevelButtonLevelOptinDeclineAll',
   'class': 'CybotCookiebotDialogBodyButton'},
  'accept_all': {'text': 'Allow all cookies',
   'id': 'CybotCookiebotDialogBodyButtonAccept',
   'class': 'CybotCookiebotDialogBodyButton'}},
 'onetrust': {'manage_my_preferences': {'text': 'Customize Settings',
   'id': 'onetrust-pc-btn-handler',
   'class': 'None'},
  'reject_all': {'text': 'Disable All',
   'id': 'onetrust-reject-all-handler',
   'class': 'None'},
  'accept_all': {'text': 'Allow All',
   'id': 'onetrust-accept-btn-handler',
   'class': 'None'},
  'confirm_my_choices': {'text': 'Confirm My Choices',
   'id': 'None',
   'class': 'save-preference-btn-handler'},
  'confirm_my_preferences': {'text': 'Confirm My Choices',
   'id': 'None',
   'class': 'save-preference-btn-handler'}},
 'stackoverflow': {'manage_my_preferences': {'text': 'Customize settings',
   'id': 'onetrust-pc-btn-handler',
   'class': 'None'},
  'reject_all': {'text': 'None',
   'id': 'onetrust-reject-all-handler',
   'class': 'None'},
  'accept_all': {'text': 'Accept all cookies',
   'id': 'onetrust-accept-btn-handler',
   'class': 'None'},
  'confirm_my_preferences': {'text': 'Confirm my choices',
   'id': 'None',
   'class': 'save-preference-btn-handler'}},
 'cnet': {'manage_my_preferences': {'text': 'Cookie Settings',
   'id': 'onetrust-pc-btn-handler',
   'class': 'None'},
  'reject_all': {'text': 'Reject All',
   'id': 'None',
   'class': 'ot-pc-refuse-all-handler'},
  'accept_all': {'text': 'Allow All',
   'id': 'accept-recommended-btn-handler',
   'class': 'None'},
  'confirm_my_preferences': {'text': 'Confirm My Choices',
   'id': 'None',
   'class': 'save-preference-btn-handler'}},
 'chicagobooth': {'manage_my_preferences': {'text': 'Manage My Cookies',
   'id': 'None',
   'class': 'toggle-modal'},
  'reject_all': {'text': 'Reject All Cookies',
   'id': 'None',
   'class': 'toggle-modal'},
  'accept_all': {'text': 'Accept All Cookies',
   'id': 'None',
   'class': 'toggle-modal'},
  'confirm_my_preferences': {'text': 'Confirm My Selections',
   'id': 'None',
   'class': 'btn'}},
 'puma': {'manage_my_preferences': {'text': 'Cookie Settings',
   'id': 'None',
   'class': 'tw-hqslau'},
  'reject_all': {'text': 'None', 'id': 'None', 'class': 'tw-hqslau'},
  'accept_all': {'text': 'I accept all', 'id': 'None', 'class': 'group'},
  'confirm_my_preferences': {'text': 'Confirm my choice',
   'id': 'None',
   'class': 'group'}},
}

In [57]:
#url_dict

In [87]:
def do_manage_and_confirm(driver, url):
    sleep_time = 1
    url_key = url.split('.')[1]
    if url_key not in url_dict:
        print("Can't handle the website ;-;")
    else:
        buttons = url_dict[url_key]
        if 'manage_my_preferences' in buttons:
            click_button(driver, buttons['manage_my_preferences'])
        sleep(sleep_time)
        if 'reject_all' in buttons:
            click_button(driver, buttons['reject_all'])
        if 'confirm_my_preferences' in buttons:
            click_button(driver, buttons['confirm_my_preferences'])
        return


            
urls = ["https://dl.acm.org/doi/10.1145/3313831.3376321",
       "https://www.onetrust.com",
       "https://www.stackoverflow.com/questions/31212059/its-possible-to-do-an-xhr-call-and-render-the-output-with-selenium",
       "https://www.cnet.com",
        "https://www.chicagobooth.edu/offerings/taking-booth-classes",
        "https://us.puma.com/us/en",
        "https://www.jamieoliver.com"
        
       ]
driver = webdriver.Firefox()
for u in urls:
    sleep(5)
    driver.get(u)
    sleep(1.5)
    t0 = time.time()
    do_manage_and_confirm(driver, u)
    t1 = time.time()
    print(u, " takes ", t1 - t0, " seconds to manage and confirm cookies \n\n\n")
        

Clicked element ID CybotCookiebotDialogDetailBodyContentCookieContainerPreference
Clicked element ID CybotCookiebotDialogBodyLevelButtonLevelOptinDeclineAll
https://dl.acm.org/doi/10.1145/3313831.3376321  takes  1.048079252243042  seconds to manage and confirm cookies 



Clicked element ID onetrust-pc-btn-handler
Clicked element ID onetrust-reject-all-handler
Clicked element text Confirm My Choices
https://www.onetrust.com  takes  1.1359748840332031  seconds to manage and confirm cookies 



Clicked element ID onetrust-pc-btn-handler
Clicked element ID onetrust-reject-all-handler
Clicked element text Confirm my choices
https://www.stackoverflow.com/questions/31212059/its-possible-to-do-an-xhr-call-and-render-the-output-with-selenium  takes  1.1573560237884521  seconds to manage and confirm cookies 



Clicked element ID onetrust-pc-btn-handler
Clicked element text Reject All
Clicked element text Confirm My Choices
https://www.cnet.com  takes  1.1740269660949707  seconds to manage and 

TimeoutException: Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:740:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19


In [59]:
#id_customize = "onetrust-pc-btn-handler"
#id_disable_all = "onetrust-reject-all-handler"
#id_allow_all = "onetrust-accept-btn-handler"
#css_performance = '[aria-label="Performance Cookies"]'
#css_marketing = '[aria-label="Targeting Cookies"]'
#css_confirm = ".save-preference-btn-handler"

In [23]:
driver.find_element(By.CSS_SELECTOR, css_performance).click()

In [22]:
driver.find_element(By.CSS_SELECTOR, css_marketing).click()

In [25]:
driver.find_element(By.CSS_SELECTOR, css_confirm).click()

This works as I can see in firefox that the cookie banner has gone away.

However, if I try to do the same on Samsung, another website that uses Onetrust, it doesn't work.

I see that a "reject all"/"decline all" button does not exist. Hence, I need to 
1. Click the "Do not Sell or share" link to make the consent banner pop up
2. Deselect the toggle for sharing data
3. Click "Confirm my choices" 

In [21]:
driver.find_element(By.PARTIAL_LINK_TEXT, "Do not Sell")

<selenium.webdriver.remote.webelement.WebElement (session="1b2a9d4e-32ec-4630-857f-9b7e1b376a2b", element="1dd36a03-70d9-4bba-9545-b9d79305a345")>

In [22]:
# I can click it
driver.find_element(By.PARTIAL_LINK_TEXT, "Do not Sell").click()

In [23]:
# Next, I need to find the switch for Selling personal Info
driver.find_element(By.CLASS_NAME, "ot-switch-nob")

<selenium.webdriver.remote.webelement.WebElement (session="1b2a9d4e-32ec-4630-857f-9b7e1b376a2b", element="799f6396-2bba-4648-b7d6-15f409226b53")>

In [24]:
# Click it
driver.find_element(By.CLASS_NAME, "ot-switch-nob").click()

In [25]:
# click the save preferences button
driver.find_element(By.CLASS_NAME, "save-preference-btn-handler.onetrust-close-btn-handler").click()

In [ ]:
driver.find_element(By.PARTIAL_LINK_TEXT, "Confirm my choices")

In [1]:
import requests

URL = "https://www.samsung.com/"
page = requests.get(URL)

print(page.text[0:500])

<!DOCTYPE html>

<html lang="en">
    
    <head>
	<!-- PWA - manifest
	<link rel="manifest" href="/etc.clientlibs/samsung/clientlibs/consumer/global/clientlib-common/resources/pwa/manifest/us.json">
	 -->
	<!-- PWA - Browser Compatible
	<script async src="/etc.clientlibs/samsung/clientlibs/consumer/global/clientlib-common/resources/pwa/js/pwacompat.js"></script>
	 -->
	<!-- <meta name="theme-color" content="#0c4da2"> -->
	<!-- Add to home screen for Safari on iOS -->
    <!-- <meta


In [38]:
f = open("samsung_page.html", "x")
f.write(page.text)
f.close()


In [42]:
URL = "https://www.onetrust.com/customers"
page = requests.get(URL)

print(page.text[0:500])

f = open("onetrust_page.html", "x")
f.write(page.text)
f.close()


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8" />
<title>Customers | OneTrust</title>
<meta name="description" content="See how OneTrust customers are using our trust intelligence platform to enhance their privacy, security, ethics, and ESG programs." />
<meta name="template" content="onetrust---base-page" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<meta name="robots" content="index, follow" />
<link rel="icon" href="/favicon.svg" />
<link rel="cano


FileExistsError: [Errno 17] File exists: 'onetrust_page.html'

In [169]:
url_html_pages = {} # URL : HTML page
url_banners = {} # URL : banner code
url_gpt_response = {} # URL : gpt_response
url_buttons = {} # URL : [buttons]

In [244]:
URL = "https://www.samsung.com/" #Decent I guess but isn't sure about "do not sell"
#URL = "https://www.chicagobooth.edu/offerings/taking-booth-classes" Perfect
#URL = "https://www.cnet.com" Chefs kiss
#URL = "https://www.stackoverflow.com/questions/31212059/its-possible-to-do-an-xhr-call-and-render-the-output-with-selenium" Perfect
#URL = "https://dl.acm.org/doi/10.1145/3313831.3376321" DOESNT WORK BUT THAT MIGHT BE OKAY
#URL = "https://www.jamieoliver.com" works
URLS = ["https://www.samsung.com/", "https://www.chicagobooth.edu/offerings/taking-booth-classes", "https://www.cnet.com", "https://www.stackoverflow.com/questions/31212059/its-possible-to-do-an-xhr-call-and-render-the-output-with-selenium", "https://dl.acm.org/doi/10.1145/3313831.3376321", "https://www.jamieoliver.com", "https://www.cava.com/", 
        "https://www.babiesrus.com/", "https://www.plannedparenthood.org"]
for url in URLS:
    url_key = url.split('/')
    print(url_key[2])

www.samsung.com
www.chicagobooth.edu
www.cnet.com
www.stackoverflow.com
dl.acm.org
www.jamieoliver.com
www.cava.com
www.babiesrus.com
www.plannedparenthood.org


In [238]:
def get_html_page(url_key, URL):
    html_source = None
    if url_key not in url_html_pages:
        driver = webdriver.Firefox()
        driver.get(URL)
        sleep(1.5)
        html_source = driver.page_source
        url_html_pages[url_key] = html_source
    else:
        html_source = url_html_pages[url_key]
    return html_source

def take_out_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    # Iterate over each <p> tag
    
    for p_tag in soup.find_all(['p', 'img', 'div', 'th', 'tr', 'td']):
        # Check for nested <a> and <button> tags
        nested_a = p_tag.find_all('a')
        nested_button = p_tag.find_all('button')
        
        if nested_a or nested_button:
            # If functional tags are found, extract them before removing <p>
            for tag in nested_a + nested_button:
                p_tag.insert_before(tag.extract())
        # Remove the <p> tag if it's now empty or if it only contains text
        if not p_tag.contents or all(isinstance(c, str) for c in p_tag.contents):
            p_tag.decompose()
    return str(soup)

def get_banner(url_key, html_source):
    banner = None
    if url_key not in url_banners:
        soup = BeautifulSoup(html_source, 'html.parser')
        keywords = ['cookie']
        divs = soup.find_all('div')
        # Find the highest div in the hierarchy
        topmost_div = None
        for div in divs:
            classes = div.get('class', [])
            ids = div.get('id', '')
            arias = div.get('aria-label', '')
            classes.append(ids)
            classes.append(arias)
            #print(classes)
            if any(keyword in classy.lower() for keyword in keywords for classy in classes):
                if (len(str(topmost_div)) < 7000) and (topmost_div is None or len(div.find_parents("div")) < len(topmost_div.find_parents("div"))):
                    topmost_div = div
        banner = str(topmost_div)
        banner = take_out_text(banner)
        url_banners[url_key] = banner
    else:
        banner = url_banners[url_key]
    return banner

def parse_response2(response):
    button_type = re.findall(r'\[type: ([^]]*)\]', response)
    button_text = re.findall(r'\[text: ([^]]*)\]', response)
    button_id = re.findall(r'\[id: ([^]]*)\]', response)
    button_class = re.findall(r'\[class: ([^]]*)\]', response)
    returned_dict = {}
    for i in range(len(button_type)):
        if button_text[i] != "None":
            returned_dict[button_type[i]] = {"text" : button_text[i],
                                            "id" : button_id[i],
                                            "class" : button_class[i]}
    return returned_dict

def get_gpt_response_and_buttons(url_key, topmost_div):
    if url_key in url_gpt_response and url_key in url_buttons:
        gpt_output = url_gpt_response[url_key]
        buttons = url_buttons[url_key]
        return (gpt_output, buttons)
    button_to_prompt = {
                        'manage_my_preferences' : 'open up a modal or display that allows the user to set more granular preferences',
                        'reject_all' : 'reject all cookies or accept only necessary/essential cookies on the site, and opt-out of tracking and data sharing',
                        'accept_all' : 'accept all cookies and consent to tracking on the website',
                        #'confirm_my_preferences' : 'confirm and save my preferences after they have selected them on the site'
                       }
    prompty = " Here is a cookie banner on a website \n " + topmost_div + " \n Which HTML element corresponds to the following types: \n "
    for button_type in button_to_prompt:
        p = f"{button_type}: allows the user to {button_to_prompt[button_type]} \n"
        prompty += p
    prompty += "And if there is no element that seems to correspond to the type (accept_all, reject_all, manage_my_preferences), please leave it out."
    prompts = [{"role": "user", "content": prompty}]
    response = client.chat.completions.create(
              model="gpt-4",
              messages=prompts
            )
    gpt_output = response.choices[0].message.content
    url_gpt_response[url_key] = gpt_output
    prompts.append({"role": "assistant", "content": gpt_output})
    formatting_prompt = "Please return outputs in the following format: [type: insert_type_here] [text: insert_text_here] [id: insert_id_here] [class: insert_class_here], where text, id, and class refer to the text contained within the element, id, and class of the HTML element, for example '[type: accept_all] [text: Accept] [id: onetrust-pc-btn-handler] [class: toggle-modal]' If any of these are not available, just enter 'None' \n "
    prompts.append({"role": "user", "content": formatting_prompt})
    response2 = client.chat.completions.create(
            model = "gpt-4",
            messages = prompts
        )
    buttons = parse_response2(response2.choices[0].message.content)
    url_buttons[url_key] = buttons
    return (gpt_output, buttons)
    

In [247]:
url = URLS[8] # get the url
url_key = url.split('/')[2] # get the key
html = get_html_page(url_key, url) # get the html
banner = get_banner(url_key, html) # get banner
gpt_response, buttons = get_gpt_response_and_buttons(url_key, banner) # get gpt_response and buttons
print(buttons)

{'manage_my_preferences': {'text': 'Cookie Settings', 'id': 'openCookieSettingsFromBanner', 'class': 'None'}, 'accept_all': {'text': 'Allow all', 'id': 'None', 'class': 'button theme-blue'}}


In [248]:
def remove_key(url_key):
    for d in [url_html_pages, url_banners, url_gpt_response, url_buttons]:
        d.pop(url_key)
        
url_buttons

{'www.samsung.com': {'reject_all': {'text': 'Do not Sell or Share My Personal Information',
   'id': 'None',
   'class': 'None'},
  'accept_all': {'text': 'Accept',
   'id': 'None',
   'class': 'cta cta--contained cta--emphasis'}},
 'www.chicagobooth.edu': {'manage_my_preferences': {'text': 'Manage My Cookies',
   'id': 'None',
   'class': 'toggle-modal'},
  'accept_all': {'text': 'Accept Cookies',
   'id': 'None',
   'class': 'btn btn-lg btn-maroon accept-cookies'}},
 'www.cnet.com': {'manage_my_preferences': {'text': 'Cookie Settings',
   'id': 'onetrust-pc-btn-handler',
   'class': 'None'}},
 'www.stackoverflow.com': {'manage_my_preferences': {'text': 'Customize settings',
   'id': 'onetrust-pc-btn-handler',
   'class': 'None'},
  'reject_all': {'text': 'Necessary cookies only',
   'id': 'onetrust-reject-all-handler',
   'class': 'None'},
  'accept_all': {'text': 'Accept all cookies',
   'id': 'onetrust-accept-btn-handler',
   'class': 'None'}},
 'dl.acm.org': {'accept_all': {'text'